# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# from plotly.subplots import make_subplots
import requests
import json

In [2]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['ig_user_id']
access_token = credentials['access_token']
group_id = credentials['STEMM_group_id']

# version 1

In [11]:
def get_group_feed(credentials_json='credentials.json', group_id_key='STEMM_group_id', 
    access_token_key='access_token',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """
    Parameters:
        - credentials.json : JSON file containing the required credentials:
            - group_id for the Facebook group.
            - access_token for the user.
        - group_id_key and access_token_key (str): Keys to the relevant group_id and access_token in the 
            JSON file.
    Returns:
        - 
    """
    # Retrieve credentials
    with open("credentials.json") as f:
        credentials = json.load(f)
    access_token = credentials[access_token_key]
    group_id = credentials[group_id_key]

    # Make API request:
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{group_id}/feed?'
    url = url_without_token + f'&access_token={access_token}'
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while page < 2:
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data']),
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
            return df_list, response_json_dict
        earliest_end_time = response_json_dict[page]['data'][0]['updated_time']
        since_parameter = datetime.strptime(earliest_end_time, "%Y-%m-%dT%H:%M:%S%z")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    return df, response_json_dict


df, response_json_dict = get_group_feed()

https://graph.facebook.com/v15.0/797362807656653/feed?
Requesting page 1...
	Response status code:  200
since_parameter:  2023-01-11 08:01:28+00:00
Number of days of data: 2


In [12]:
df

,updated_time,message,id
0,2023-01-11T08:01:28+0000,There was a time when my only exercise was run...,797362807656653_1233988560660740
1,2022-12-27T20:49:40+0000,With power comes responsibility.\nIn the fitne...,797362807656653_1224710061588590


In [16]:
df2, response_json_dict2 = get_group_feed(credentials_json='credentials.json', group_id_key='SSC_group_id', 
    access_token_key='am_fb_access_token')

https://graph.facebook.com/v15.0/2139238999669147/feed?
Requesting page 1...
	Response status code:  200
since_parameter:  2023-03-16 18:50:59+00:00
Number of days of data: 25


In [17]:
df2

,updated_time,message,id
0,2023-03-16T18:50:59+0000,If you’re constantly worried about calories an...,2139238999669147_3367071606885874
1,2023-03-15T20:56:26+0000,You’ve heard the saying ‘a dog is for life - n...,2139238999669147_3366477613611940
2,2023-03-16T00:52:42+0000,Calories in versus calories out (how do I over...,2139238999669147_3366060270320341
3,2023-03-15T02:45:56+0000,"The humble back photo 🔙\n\nIn my opinion, usua...",2139238999669147_3365771280349240
4,2023-03-13T18:46:00+0000,It would actually break your heart to read my ...,2139238999669147_3365106907082344
5,2023-03-13T07:55:10+0000,@everyone would love your help with the live t...,2139238999669147_3364783420448026
6,2023-03-13T08:30:30+0000,Have your ever tried on a pair of shorts - jus...,2139238999669147_3364475920478776
7,2023-03-10T22:03:08+0000,Do you know your why?,2139238999669147_3362414344018267
8,2023-03-09T18:57:21+0000,Being organised is the key to success 🔑\n\nThi...,2139238999669147_3362409990685369
9,2023-03-09T08:14:06+0000,@everyone any Sydney Siders on here looking to...,2139238999669147_3361852350741133


# Version 2

In [21]:
def get_group_feed(credentials_json='credentials.json', group_id_key='STEMM_group_id', 
    access_token_key='access_token',
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """
    Parameters:
        - credentials.json : JSON file containing the required credentials:
            - group_id for the Facebook group.
            - access_token for the user.
        - group_id_key and access_token_key (str): Keys to the relevant group_id and access_token in the 
            JSON file.
    Returns:
        - 
    """
    # Retrieve credentials
    with open("credentials.json") as f:
        credentials = json.load(f)
    access_token = credentials[access_token_key]
    group_id = credentials[group_id_key]

    # Make API request:
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{group_id}/feed?fields=updated_time,message,permalink_url'
    url = url_without_token + f'&access_token={access_token}'
    print(url_without_token)
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while page < 2:
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data']),
                ], axis=1)
            )
            earliest_end_time = response_json_dict[page]['data'][0]['updated_time']
            since_parameter = datetime.strptime(earliest_end_time, "%Y-%m-%dT%H:%M:%S%z")
            print('since_parameter: ',since_parameter)
        except:
            print('No data in request response for page', page)
            return df_list, response_json_dict

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
    except:
        df = df_list 
    return df, response_json_dict


df2, response_json_dict2 = get_group_feed(credentials_json='credentials.json', group_id_key='SSC_group_id', 
    access_token_key='am_fb_access_token')
df2

https://graph.facebook.com/v15.0/2139238999669147/feed?fields=updated_time,message,permalink_url
Requesting page 1...
	Response status code:  200
since_parameter:  2023-03-16 18:50:59+00:00
Number of days of data: 25


,updated_time,message,permalink_url,id
0,2023-03-16T18:50:59+0000,If you’re constantly worried about calories an...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3367071606885874
1,2023-03-15T20:56:26+0000,You’ve heard the saying ‘a dog is for life - n...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3366477613611940
2,2023-03-16T00:52:42+0000,Calories in versus calories out (how do I over...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3366060270320341
3,2023-03-15T02:45:56+0000,"The humble back photo 🔙\n\nIn my opinion, usua...",https://www.facebook.com/groups/strongandsassy...,2139238999669147_3365771280349240
4,2023-03-13T18:46:00+0000,It would actually break your heart to read my ...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3365106907082344
5,2023-03-13T07:55:10+0000,@everyone would love your help with the live t...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3364783420448026
6,2023-03-13T08:30:30+0000,Have your ever tried on a pair of shorts - jus...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3364475920478776
7,2023-03-10T22:03:08+0000,Do you know your why?,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3362414344018267
8,2023-03-09T18:57:21+0000,Being organised is the key to success 🔑\n\nThi...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3362409990685369
9,2023-03-09T08:14:06+0000,@everyone any Sydney Siders on here looking to...,https://www.facebook.com/groups/strongandsassy...,2139238999669147_3361852350741133


# *End of Page*